In [2]:
import requests
import pandas as pn
import json
from datetime import datetime as dt

### Finding number of Astronaut in ISS currently

In [3]:
api_call=requests.get('http://api.open-notify.org/astros.json')
print(api_call.status_code)

200


In [4]:
#this is how api response looks
print(json.dumps(api_call.json()))

{"message": "success", "number": 7, "people": [{"craft": "ISS", "name": "Sergey Ryzhikov"}, {"craft": "ISS", "name": "Kate Rubins"}, {"craft": "ISS", "name": "Sergey Kud-Sverchkov"}, {"craft": "ISS", "name": "Mike Hopkins"}, {"craft": "ISS", "name": "Victor Glover"}, {"craft": "ISS", "name": "Shannon Walker"}, {"craft": "ISS", "name": "Soichi Noguchi"}]}


In [5]:
#to print the number of astronauts
print('Number',api_call.json()['number'])

Number 7


### Name of the People and their  crafts

In [6]:
print(pn.DataFrame(api_call.json()['people']))

  craft                  name
0   ISS       Sergey Ryzhikov
1   ISS           Kate Rubins
2   ISS  Sergey Kud-Sverchkov
3   ISS          Mike Hopkins
4   ISS         Victor Glover
5   ISS        Shannon Walker
6   ISS        Soichi Noguchi


### ISS coming over my head in next 24hrs

In [7]:
#we will call 2 API here, one to fetch Latitude and longitude based on the city
city='hazaribagh'
geo_api=requests.get('http://api.positionstack.com/v1/forward?access_key=58e157cf01d0ed393968ddc1a04e11ef&query='+city+'')

#Second API to fetch ISS time and duration of pass on that latitude and longitude location or over that City
#we will use above api's latitude and long info on API2
parameters={'lat':geo_api.json()['data'][0]['latitude'],'lon':geo_api.json()['data'][0]['longitude']}
api2=requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

if geo_api.status_code==200 and api2.status_code==200:#checking the connection with API
    print('Connected')
else:
    print('Not Connected')

Connected


In [8]:
api2.json()#you can see the response 

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1613380878,
  'latitude': 23.99507,
  'longitude': 85.36109,
  'passes': 5},
 'response': [{'duration': 650, 'risetime': 1613384314},
  {'duration': 184, 'risetime': 1613390323},
  {'duration': 566, 'risetime': 1613432557},
  {'duration': 619, 'risetime': 1613438327},
  {'duration': 128, 'risetime': 1613444434}]}

In [18]:
df1=pn.DataFrame(api2.json()['response'])#used pandas to show the Response of api2 in Tabular form
df1

,duration,risetime
0,650,1613384314
1,184,1613390323
2,566,1613432557
3,619,1613438327
4,128,1613444434


In [19]:
#risetime need to be converted to human read time format
time_list=[]
for i in api2.json()['response']:
    print(dt.fromtimestamp(i['risetime']))
    time_list.append(dt.fromtimestamp(i['risetime']))

d={'Time':time_list}
dt_df=pn.DataFrame(d)

2021-02-15 15:48:34
2021-02-15 17:28:43
2021-02-16 05:12:37
2021-02-16 06:48:47
2021-02-16 08:30:34


In [20]:
#merging two dataframe
df3=pn.merge(df1,dt_df,right_index=True,left_index=True)
df3

,duration,risetime,Time
0,650,1613384314,2021-02-15 15:48:34
1,184,1613390323,2021-02-15 17:28:43
2,566,1613432557,2021-02-16 05:12:37
3,619,1613438327,2021-02-16 06:48:47
4,128,1613444434,2021-02-16 08:30:34


In [21]:
#splitting the Time to Date and Clock time for more convienient reading
df3['Date']=[d.date() for d in df3['Time']]
df3['Clock time']=[d.time() for d in df3['Time']]
df3

,duration,risetime,Time,Date,Clock time
0,650,1613384314,2021-02-15 15:48:34,2021-02-15,15:48:34
1,184,1613390323,2021-02-15 17:28:43,2021-02-15,17:28:43
2,566,1613432557,2021-02-16 05:12:37,2021-02-16,05:12:37
3,619,1613438327,2021-02-16 06:48:47,2021-02-16,06:48:47
4,128,1613444434,2021-02-16 08:30:34,2021-02-16,08:30:34


In [22]:
#Adding Duration in min 
df3['Duration in min']=[d/60 for d in df3['duration']]
df3

,duration,risetime,Time,Date,Clock time,Duration in min
0,650,1613384314,2021-02-15 15:48:34,2021-02-15,15:48:34,10.833333
1,184,1613390323,2021-02-15 17:28:43,2021-02-15,17:28:43,3.066667
2,566,1613432557,2021-02-16 05:12:37,2021-02-16,05:12:37,9.433333
3,619,1613438327,2021-02-16 06:48:47,2021-02-16,06:48:47,10.316667
4,128,1613444434,2021-02-16 08:30:34,2021-02-16,08:30:34,2.133333


In [23]:
#converting df3 to human readable data and time data of ISS passing
time_table=df3[['Date','Clock time','Duration in min']]
time_table

,Date,Clock time,Duration in min
0,2021-02-15,15:48:34,10.833333
1,2021-02-15,17:28:43,3.066667
2,2021-02-16,05:12:37,9.433333
3,2021-02-16,06:48:47,10.316667
4,2021-02-16,08:30:34,2.133333
